In [9]:
import pandas as pd

leases_df = pd.read_csv("../data/Leases.csv")

remove_features = [
    "direct_available_space",
    "direct_availability_proportion",
    "direct_internal_class_rent",
    "direct_overall_rent",
    "sublet_available_space",
    "sublet_availability_proportion",
    "sublet_internal_class_rent",
    "sublet_overall_rent"
]

leases_df = leases_df.drop(remove_features, axis=1)
leases_df

,year,quarter,monthsigned,market,building_name,building_id,address,region,city,state,...,internal_market_cluster,costarID,space_type,CBD_suburban,RBA,available_space,availability_proportion,internal_class_rent,overall_rent,leasing
0,2018,Q1,1.0,Atlanta,10 Glenlake North Tower,Atlanta_Central Perimeter_Atlanta_10 Glenlake ...,10 Glenlake Pky NE,South,Atlanta,GA,...,NaN,445509,Relet,Suburban,101140416.0,20239067.0,0.200109,27.655891,24.345689,1205126.0
1,2018,Q1,1.0,Atlanta,100 City View,Atlanta_Northwest_Atlanta_100 City View_3330 C...,3330 Cumberland Blvd,South,Atlanta,GA,...,NaN,436994,Relet,Suburban,101140416.0,20239067.0,0.200109,27.655891,24.345689,1205126.0
2,2018,Q1,1.0,Atlanta,1000 Parkwood,Atlanta_Northwest_Atlanta_1000 Parkwood_1000 P...,1000 Parkwood Cir SE,South,Atlanta,GA,...,NaN,434890,Relet,Suburban,101140416.0,20239067.0,0.200109,27.655891,24.345689,1205126.0
3,2018,Q1,1.0,Atlanta,1100 Circle 75,Atlanta_Northwest_Atlanta_1100 Circle 75_1100 ...,1100 Circle 75 Pky SE,South,Atlanta,GA,...,NaN,434720,Relet,Suburban,65810449.0,12728989.0,0.193419,18.560886,24.345689,715742.0
4,2018,Q1,1.0,Atlanta,1200 Ashwood,Atlanta_Central Perimeter_Atlanta_1200 Ashwood...,1200 Ashwood Pky,South,Atlanta,GA,...,NaN,437562,Relet,Suburban,101140416.0,20239067.0,0.200109,27.655891,24.345689,1205126.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194680,2024,Q4,12.0,Tampa,Building II,Tampa_Westshore_Tampa_Building II_3111 W Dr Ma...,3111 W Dr Martin Luther King Blvd,South,Tampa,FL,...,Hillsborough,389404,NaN,Suburban,25106444.0,5772868.0,0.230000,32.980000,29.460000,770384.0
194681,2024,Q4,12.0,Tampa,Midtown East,Tampa_Westshore_Tampa_Midtown East_1 Midtown Pl,1 Midtown Pl,South,Tampa,FL,...,Hillsborough,612255,NaN,Suburban,25106444.0,5772868.0,0.230000,32.980000,29.460000,770384.0
194682,2024,Q4,12.0,Tampa,Park Tower,Tampa_Tampa CBD_Tampa_Park Tower_400 N Tampa St,400 N Tampa St,South,Tampa,FL,...,Hillsborough,380519,Relet,CBD,29139850.0,4891754.0,0.168000,25.600000,29.460000,328947.0
194683,2024,Q4,12.0,Tampa,Rivergate Tower,Tampa_Tampa CBD_Tampa_Rivergate Tower_400 N As...,400 N Ashley Dr,South,Tampa,FL,...,Hillsborough,380475,NaN,CBD,25106444.0,5772868.0,0.230000,32.980000,29.460000,770384.0
